In [78]:
from docx import Document
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
import os

In [79]:
# División de los chunks, 1 por cada párrafo
def get_paragraph_chunks(doc_path):
    doc = Document(doc_path)
    paragraph_chunks = [paragraph.text for paragraph in doc.paragraphs if paragraph.text]
    return paragraph_chunks

#Embbedding de los chunks
def generate_vectors_with_metadata(paragraph_chunks):
    model = SentenceTransformer("all-MiniLM-L6-v2")  # Cargar modelo pre-entrenado
    
    vectors_with_metadata = []
    for idx, chunk in enumerate(paragraph_chunks):
        # Generar vector de representación para cada párrafo
        vector = model.encode(chunk)
        
        metadata = {"text": chunk, "paragraph_length": len(chunk)}
        
        vectors_with_metadata.append({"id": f"doc1#chunk{idx+1}", "values": vector, "metadata": metadata})
    
    return vectors_with_metadata

# Obtener los chunks de párrafos del documento
doc_path = 'C:\\Users\\rigel\\Desktop\\AIEngineer\\rag_llm\\data\\documento.docx'
paragraph_chunks = get_paragraph_chunks(doc_path)

# Generar vectores con metadatos para los chunks de párrafos
vectors_with_metadata = generate_vectors_with_metadata(paragraph_chunks)
print(len(vectors_with_metadata))
print(vectors_with_metadata)





5
[{'id': 'doc1#chunk1', 'values': array([ 6.29131719e-02, -3.20602246e-02, -1.99455395e-02,  2.20068041e-02,
       -1.54253589e-02, -7.27200881e-02,  3.43870334e-02,  2.60197390e-02,
        4.78636883e-02,  8.43962952e-02, -6.49563968e-03, -6.18437082e-02,
        3.27206776e-02,  2.27239318e-02, -4.74721529e-02, -8.24746564e-02,
       -5.46359792e-02, -2.82873567e-02, -2.54074484e-02,  1.44576773e-01,
        7.72100538e-02, -2.07235329e-02, -3.31582278e-02,  1.48509130e-01,
       -6.91467002e-02,  7.96619151e-03, -7.40731834e-03, -1.38086034e-02,
       -6.66001961e-02, -2.18766965e-02,  4.26250249e-02,  4.15589511e-02,
        1.05254404e-01,  1.11198025e-02, -2.14183442e-02,  3.58321741e-02,
        1.69898439e-02, -2.43382286e-02,  8.19417823e-04, -2.14992948e-02,
       -5.16877808e-02, -8.45951959e-03,  3.31684798e-02, -1.69316623e-02,
       -8.33887607e-02, -6.58330321e-02, -2.45625973e-02,  1.23141836e-02,
       -2.79254243e-02, -2.20821314e-02, -2.98723541e-02, -6.1657

In [42]:
# Inicializar Pinecone, db vectorial
pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))
index = pc.Index(os.environ.get('PINECONE_INDEX'))
index.upsert(vectors_with_metadata)

{'upserted_count': 5}

In [96]:
#Prueba de busqueda de similitud
def find_match(input):
    model = SentenceTransformer("all-MiniLM-L6-v2")
    pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))
    index = pc.Index(os.environ.get('PINECONE_INDEX'))
    input_em = model.encode(input).tolist()
    result = index.query(vector=input_em, top_k=1, includeMetadata=True)

    return result["matches"][0]["metadata"]['text']


In [97]:
resultado = find_match("zara")
print(resultado)

Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.


In [82]:
import cohere


In [83]:
#Prueba de generacion de respuesta con LLM

co = cohere.Client(os.environ.get('COHERE_API_KEY'))


In [86]:
pregunta_usuario = "¿quien es zara?"

In [94]:
def generate_answers(texto, pregunta_usuario):

  prompt = f"""
  responda a la pregunta en una sola oración, en español.

  **User:** {pregunta_usuario}

  **Respuesta:** {texto}

  A continuación, por favor, proporcione su respuesta:
  """

  response = co.chat(
      message=prompt,
      model="command",
      temperature=0.2,
  )

  return response.text



In [95]:

respuesta_generada = generate_answers(texto, pregunta_usuario)
print(respuesta_generada)

Zara es un personaje ficticio en un cuento espacial sobre dos civilizaciones alienígenas en la galaxia Zenthoria que buscan un misterioso artefacto antiguo que puede contener la clave de la paz entre los dos galaxias.
